<a href="https://colab.research.google.com/github/ptmhoang97/train_dataset/blob/main/TransferLearning_GGdrive_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Sep 21 18:34:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [62]:
batch_size_pipeline = 64
num_steps_training = 50000
learning_rate_pipeline = 0.001
warmup_learning_rate_pipeline = learning_rate_pipeline/10
models_drive_path = "/content/drive/MyDrive/workspaces_tensorflow/test_resume_training"
model_name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
train_dataset = "/content/drive/MyDrive/coco2017_train_car.zip"
test_dataset = "/content/drive/MyDrive/coco2017_validation_car.zip"

# 0. Setup Paths

In [46]:
import os

In [47]:
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [48]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
    'MODEL_PATH': os.path.join(models_drive_path),
    'CHECKPOINT_PATH': os.path.join(models_drive_path, 'next_trained_model'), 
    'OUTPUT_PATH': os.path.join(models_drive_path, 'next_trained_model', 'export'), 
    'TFJS_PATH':os.path.join(models_drive_path, 'next_trained_model', 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(models_drive_path, 'next_trained_model', 'tfliteexport'), 
    'PRETRAINED_MODEL_PATH':os.path.join(models_drive_path, model_name), 
 }

In [49]:
files = {
    'PIPELINE_CONFIG':os.path.join(models_drive_path, 'next_trained_model', 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME),
    'PIPELINE_CONFIG_BASE':os.path.join(models_drive_path, model_name, 'pipeline.config'),
    'CHECKPOINT_BASE':os.path.join(models_drive_path, model_name, 'checkpoint', 'ckpt-0'),
}

In [50]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}
        print(path)

/content/drive/MyDrive/workspaces_tensorflow/test_resume_training/next_trained_model
/content/drive/MyDrive/workspaces_tensorflow/test_resume_training/next_trained_model/export
/content/drive/MyDrive/workspaces_tensorflow/test_resume_training/next_trained_model/tfjsexport
/content/drive/MyDrive/workspaces_tensorflow/test_resume_training/next_trained_model/tfliteexport


# 1. Install TF Object Dectection API

In [9]:
# https://www.tensorflow.org/install/source_windows

In [10]:
if os.name=='nt':
    !pip install wget
    import wget

In [11]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 63035, done.
remote: Counting objects: 100% (375/375), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 63035 (delta 251), reused 317 (delta 199), pack-reused 62660
Receiving objects: 100% (63035/63035), 574.63 MiB | 11.31 MiB/s, done.
Resolving deltas: 100% (43981/43981), done.


In [12]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Processing /content/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 90 kB/s 
     |████████████████████████████████| 352 kB 92.3 MB/s 
     |████████████████████████████████| 1.8 MB 65.9 MB/s 
     |████████████████████████████████| 37.1 MB 83 kB/s 
     |████████████████████████████████| 90 kB 11.8 MB/s 
     |████████████████████████████████| 211

In [13]:
print(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py'))

Tensorflow/models/research/object_detection/builders/model_builder_tf2_test.py


In [14]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2021-09-21 18:37:56.199893: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-21 18:37:56.434028: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-21 18:37:56.434798: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-09-21 18:37:56.449353: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021

In [15]:
import object_detection

# 2. Download dataset

In [16]:
!unzip {train_dataset} -d {paths['IMAGE_PATH']}

Streaming output truncated to the last 5000 lines.
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000256576.jpg  
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000465638.xml  
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000019324.xml  
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000083364.xml  
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000537289.xml  
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000203269.jpg  
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000121006.xml  
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000529314.jpg  
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000253444.jpg  
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000269888.jpg  
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000529842.jpg  
  inflating: Tensorflow/workspace/images/coco2017_train_car/000000185177.xml  
 

In [17]:
!unzip {test_dataset} -d {paths['IMAGE_PATH']}

Archive:  /content/drive/MyDrive/coco2017_validation_car.zip
   creating: Tensorflow/workspace/images/coco2017_validation_car/
  inflating: Tensorflow/workspace/images/coco2017_validation_car/000000011197.jpg  
  inflating: Tensorflow/workspace/images/coco2017_validation_car/000000379800.xml  
  inflating: Tensorflow/workspace/images/coco2017_validation_car/000000303908.xml  
  inflating: Tensorflow/workspace/images/coco2017_validation_car/000000393093.xml  
  inflating: Tensorflow/workspace/images/coco2017_validation_car/000000166664.xml  
  inflating: Tensorflow/workspace/images/coco2017_validation_car/000000365207.xml  
  inflating: Tensorflow/workspace/images/coco2017_validation_car/000000279714.xml  
  inflating: Tensorflow/workspace/images/coco2017_validation_car/000000462756.xml  
  inflating: Tensorflow/workspace/images/coco2017_validation_car/000000162858.jpg  
  inflating: Tensorflow/workspace/images/coco2017_validation_car/000000094751.xml  
  inflating: Tensorflow/workspace

# 3. Create Label Map

In [18]:
labels = [{'name':'car', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 4. Create TF records

In [19]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

Cloning into 'Tensorflow/scripts'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [20]:
train_dataset_folder = train_dataset[train_dataset.rfind("/")+1:train_dataset.rfind(".")]
test_dataset_folder = test_dataset[test_dataset.rfind("/")+1:test_dataset.rfind(".")]
print(train_dataset_folder)
print(test_dataset_folder)

coco2017_train_car
coco2017_validation_car


In [21]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], train_dataset_folder)} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], test_dataset_folder)} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 5. Copy Model Config to Training Folder

In [63]:
if os.name =='posix':
    !cp {os.path.join(files['PIPELINE_CONFIG_BASE'])} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(files['PIPELINE_CONFIG_BASE'])} {os.path.join(paths['CHECKPOINT_PATH'])}

# 6. Update Config For Transfer Learning

In [64]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [65]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [66]:
config

{'eval_config': metrics_set: "coco_detection_metrics"
 use_moving_averages: false,
 'eval_input_config': label_map_path: "PATH_TO_BE_CONFIGURED"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BE_CONFIGURED"
 },
 'eval_input_configs': [label_map_path: "PATH_TO_BE_CONFIGURED"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BE_CONFIGURED"
 }
 ],
 'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.9969999790191

In [67]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [68]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = batch_size_pipeline
pipeline_config.train_config.num_steps = num_steps_training
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.total_steps = num_steps_training
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = learning_rate_pipeline
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = warmup_learning_rate_pipeline
pipeline_config.train_config.fine_tune_checkpoint = files['CHECKPOINT_BASE']
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [69]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 7. Train the model

In [41]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [42]:
command = "python {} --model_dir={} --pipeline_config_path={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [43]:
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=/content/drive/MyDrive/workspaces_tensorflow/test_resume_training/next_trained_model --pipeline_config_path=/content/drive/MyDrive/workspaces_tensorflow/test_resume_training/next_trained_model/pipeline.config


In [44]:
!{command}

2021-09-21 18:41:27.077528: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-21 18:41:27.086044: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-21 18:41:27.086651: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-21 18:41:27.087838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-21 18:41:27.088531: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

# 8. Evaluate model

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

# 10. Zip model and copy to GG Drive

In [ ]:
import datetime
import pytz

def shorten_time(date):
    dt_year = date[0:4]
    dt_month = date[5:7]
    dt_day = date[8:10]
    dt_hr = date[11:13]
    dt_min = date[14:16]
    dt_sec = date[17:19]
    dt_full = "{}{}{}_{}{}{}".format(dt_year,dt_month,dt_day,dt_hr,dt_min,dt_sec)
    return dt_full

start_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
start_date_short = shorten_time(start_date)
print(start_date)
print(start_date_short)

In [ ]:
%cd {paths['MODEL_PATH']}

In [ ]:
export_model_path = "next_trained_model"
export_model_zip = "{}_{}.zip".format(export_model_path,start_date_short)
print(export_model_path)
print(export_model_zip)

In [ ]:
!zip -r {export_model_zip} {export_model_path}

In [ ]:
%cd "/content"

# 11. Notification via Email

In [ ]:
import datetime
import pytz

finish_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
finish_date_short = shorten_time(finish_date)

print(finish_date)
print(finish_date_short)

In [ ]:
sd_year = int(start_date_short[0:4])
sd_month = int(start_date_short[4:6])
sd_day = int(start_date_short[6:8])
sd_hour = int(start_date_short[9:11])
sd_min = int(start_date_short[11:13])
sd_sec = int(start_date_short[13:15])

fd_year = int(finish_date_short[0:4])
fd_month = int(finish_date_short[4:6])
fd_day = int(finish_date_short[6:8])
fd_hour = int(finish_date_short[9:11])
fd_min = int(finish_date_short[11:13])
fd_sec = int(finish_date_short[13:15])

In [ ]:
start = datetime.datetime(year=sd_year, month=sd_month, day=sd_day, hour=sd_hour, minute=sd_min,second=sd_sec)
end = datetime.datetime(year=fd_year, month=fd_month, day=fd_day, hour=fd_hour, minute=fd_min,second=fd_sec)
diff = end - start
print(diff)

In [ ]:
import smtplib
from email.message import EmailMessage

msg_subject = "[GG Colab] Model {}".format(start_date_short)
msg_content = "It take {} for training.".format(diff)

msg = EmailMessage()
msg['Subject'] = msg_subject
msg['From'] = '*'
msg['To']='ptmhoang97@gmail.com'
msg.set_content(msg_content)

server = smtplib.SMTP_SSL('smtp.gmail.com',465)
server.login("ptmhoang97@gmail.com","euclxykrqkkrquqz")
server.send_message(msg)
server.quit()